In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [5]:
def extract_carbon_data_from_calibration(cal_file_path):
    """
    Extract carbon data from the calibration CSV file for regression.
    
    Parameters:
    -----------
    cal_file_path : str
        Path to the calibration CSV file
    
    Returns:
    --------
    tuple: (slope, intercept, r_squared)
    """
    # Read the CSV file
    df = pd.read_csv(cal_file_path, header=None, skip_blank_lines=True)
    
    # Filter rows with Carbon data (Compound Name is 'Carbon')
    carbon_data = df[df[9] == 'Carbon']
    
    # Extract weight and area values
    weights = carbon_data[6].astype(float)
    areas = carbon_data[5].astype(float)
    
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(weights, areas)
    
    return slope, intercept, r_value**2

def quantify_carbon_in_samples(data_file_path, slope, intercept):
    """
    Quantify carbon in samples using the calibration line.
    
    Parameters:
    -----------
    data_file_path : str
        Path to the data CSV file
    slope : float
        Slope of the calibration line
    intercept : float
        Intercept of the calibration line
    
    Returns:
    --------
    pandas.DataFrame: Quantified carbon data
    """
    # Read the data file
    df = pd.read_csv(data_file_path, header=None, skip_blank_lines=True)
    
    # Prepare results list
    results = []
    
    # Process each unique primary file
    unique_files = df[0].unique()
    
    for file_num in unique_files:
        # Filter rows for this file with Carbon data
        file_data = df[(df[0] == file_num) & (df[9] == 'Carbon')]
        
        # Skip if no carbon data for this file
        if file_data.empty:
            continue
        
        # Extract area for carbon peak
        carbon_area = file_data[5].astype(float).values[0]
        
        # Calculate weight using calibration line (y = mx + b)
        # Rearrange to x = (y - b) / m
        carbon_weight_mg = (carbon_area - intercept) / slope
        carbon_weight_ug = carbon_weight_mg * 1000
        
        results.append({
            'Primary File': file_num,
            'Carbon Area [mV.s]': carbon_area,
            'Carbon Weight [mg]': carbon_weight_mg,
            'Carbon Weight [µg]': carbon_weight_ug
        })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

def main(cal_file_path, data_file_path):
    """
    Main function to perform carbon quantification.
    
    Parameters:
    -----------
    cal_file_path : str
        Path to the calibration CSV file
    data_file_path : str
        Path to the data CSV file
    """
    # Get calibration line parameters
    slope, intercept, r_squared = extract_carbon_data_from_calibration(cal_file_path)
    
    print("Calibration Line Parameters:")
    print(f"Slope: {slope}")
    print(f"Intercept: {intercept}")
    print(f"R-squared: {r_squared}")
    
    # Quantify carbon in samples
    carbon_quantification = quantify_carbon_in_samples(data_file_path, slope, intercept)
    
    # Display results
    print("\nCarbon Quantification Results:")
    print(carbon_quantification)
    
    # Optional: Save to CSV
    carbon_quantification.to_csv('carbon_quantification_results.csv', index=False)
    
    return carbon_quantification

In [6]:
cal_file = 'Katz_20141210_P-vent_CTO_Test_cal.csv'
data_file = 'Katz_20141210_P-vent_CTO_Test_data.csv'
results = main(cal_file, data_file)

Calibration Line Parameters:
Slope: 6907.35253728295
Intercept: 1.8830308802221225
R-squared: 0.9995558373076884

Carbon Quantification Results:
   Primary File  Carbon Area [mV.s]  Carbon Weight [mg]  Carbon Weight [µg]
0           002           2706.5874            0.391569          391.568890
1           003           1409.3142            0.203758          203.758410
2           004           6444.3133            0.932692          932.691684
3           005           3270.2968            0.473179          473.178943
4           015             53.0818            0.007412            7.412213
5           016             37.5432            0.005163            5.162639
6           017            529.9675            0.076453           76.452514
7           018            142.4251            0.020347           20.346735
8           019           2300.6468            0.332800          332.799543
9           020            374.2347            0.053907           53.906568
10          021    